In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Fetch Data
def fetch_data(symbol, start_date, end_date):
    # Assuming data is in a CSV file for simplicity
    df = pd.read_csv(f'{symbol}.csv')  # Replace with the path to your CSV file
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df.loc[start_date:end_date]
    return df

# Step 2: Calculate Indicators
def macd(df, fast_period, slow_period, signal_period):
    df['EMA_fast'] = df['Close'].ewm(span=fast_period, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow_period, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['Signal'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['Signal']
    return df

def bollinger_bands(df, bb_period, num_std_dev):
    df['MA'] = df['Close'].rolling(window=bb_period).mean()
    df['BB_Upper'] = df['MA'] + (df['Close'].rolling(window=bb_period).std() * num_std_dev)
    df['BB_Lower'] = df['MA'] - (df['Close'].rolling(window=bb_period).std() * num_std_dev)
    return df

def atr(df, atr_period):
    df['High-Low'] = df['High'] - df['Low']
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    df['ATR'] = df['TrueRange'].rolling(window=atr_period).mean()
    return df

# Step 3: Backtesting Function
def backtest(df, fast_period, slow_period, signal_period, bb_period, num_std_dev, atr_period):
    df = macd(df, fast_period, slow_period, signal_period)
    df = bollinger_bands(df, bb_period, num_std_dev)
    df = atr(df, atr_period)

    initial_capital = 100000
    capital = initial_capital
    position = 0
    portfolio_value = []

    for i in range(1, len(df)):
        if df['MACD'][i-1] < df['Signal'][i-1] and df['MACD'][i] > df['Signal'][i] and df['Close'][i] < df['BB_Lower'][i]:
            if position == 0:
                position = capital / df['Close'][i]
                capital = 0
                print(f"Buy at {df.index[i]}: {df['Close'][i]}")
        elif df['MACD'][i-1] > df['Signal'][i-1] and df['MACD'][i] < df['Signal'][i] and df['Close'][i] > df['BB_Upper'][i]:
            if position > 0:
                capital = position * df['Close'][i]
                position = 0
                print(f"Sell at {df.index[i]}: {df['Close'][i]}")

        portfolio_value.append(capital + position * df['Close'][i])

    df['Portfolio Value'] = portfolio_value
    total_return = (capital + position * df['Close'].iloc[-1] - initial_capital) / initial_capital
    print(f"Total Return: {total_return * 100:.2f}%")

    return df

# Step 4: Running the Backtest and Plotting
if __name__ == "__main__":
    # Parameters
    fast_period = 12
    slow_period = 26
    signal_period = 9
    bb_period = 20
    num_std_dev = 2
    atr_period = 14

    # Fetch the data
    df = fetch_data('AAPL', '2020-01-01', '2023-01-01')

    # Run the backtest
    df = backtest(df, fast_period, slow_period, signal_period, bb_period, num_std_dev, atr_period)

    # Plotting the portfolio value
    plt.figure(figsize=(14,7))
    plt.plot(df.index, df['Portfolio Value'], label='Portfolio Value')
    plt.plot(df.index, df['Close'], label='Close Price')
    plt.title('Backtesting Results')
    plt.legend()
    plt.show()
